In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("PySpark Access Logs") \
    .master("spark://172.29.16.102:7077") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/22 17:54:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pandas as pd

In [4]:
logs_df = pd.read_csv('listpw.csv')

In [5]:
logs_df.head()

,date,time,user,password
0,3/23/2022,22:54:06,NaN,NaN
1,3/23/2022,22:54:26,user,1
2,3/23/2022,22:54:49,NaN,NaN
3,3/23/2022,22:55:49,NaN,NaN
4,3/23/2022,22:56:24,NaN,NaN


In [6]:
logs_df.count()

date        15211
time        15211
user        14234
password    10634
dtype: int64

In [7]:
users = logs_df['user'].to_list()

In [8]:
sc = spark.sparkContext

In [9]:
log_rdd = sc.parallelize(users)

In [10]:
log_rdd.getNumPartitions()

2

In [15]:
log_rdd.takeSample(num = 5, withReplacement = False)

[nan, nan, 'root', 'root', 'root']

In [16]:
def mapper(user):
    return (str(user), 1)

In [17]:
mapped_rdd = log_rdd.map(mapper)

In [20]:
mapped_rdd.takeSample(num = 5, withReplacement = False)

[('admin', 1), ('root', 1), ('nan', 1), ('root', 1), ('root', 1)]

In [21]:
def reducer(count1, count2):
    return count1 + count2

In [22]:
route_counts = mapped_rdd.reduceByKey(reducer)

In [23]:
route_counts.takeSample(num = 5, withReplacement = False)

[('db2inst1', 2),
 ('micom', 2),
 ('spark', 1),
 ('plcmspip', 10),
 ('monitor', 12)]

In [24]:
sorted_routes = route_counts.sortBy(lambda x: x[1], ascending=False)

In [27]:
len(sorted_routes.collect())

396

In [26]:
for route, count in sorted_routes.collect():
    print(f"User: {route}, Access Count: {count}")

User: root, Access Count: 9644
User: admin, Access Count: 1476
User: nan, Access Count: 977
User: user, Access Count: 417
User: Administrator, Access Count: 229
User: identification, Access Count: 136
User: 111111, Access Count: 120
User: support, Access Count: 109
User: test, Access Count: 108
User: ubnt, Access Count: 103
User: 1234, Access Count: 74
User: 123321, Access Count: 71
User: guest, Access Count: 56
User: ftpuser, Access Count: 56
User: oracle, Access Count: 47
User: adm, Access Count: 45
User: ftp, Access Count: 45
User: pi, Access Count: 45
User: administrator, Access Count: 44
User: operator, Access Count: 38
User: super, Access Count: 37
User: ubuntu, Access Count: 36
User: default, Access Count: 34
User: manager, Access Count: 34
User: master, Access Count: 34
User: usuario, Access Count: 33
User: nagios, Access Count: 33
User: 123, Access Count: 31
User: Admin, Access Count: 29
User: mos, Access Count: 22
User: sshd, Access Count: 21
User: user1, Access Count: 21
Use

In [28]:
spark.stop()